<h1><center>Battle of Neighborhoods - Capstone</center></h1>

### Importing Libraries

In [1]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!pip install folium
import folium #import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Extracting required dataset

In [2]:
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

In [4]:
df_ppd.head(10)

,{79A74E21-D11E-1289-E053-6B04A8C01627},770000,2018-09-25 00:00,SK7 1AR,D,N,F,5,Unnamed: 8,OAK MEADOW,BRAMHALL,STOCKPORT,STOCKPORT.1,GREATER MANCHESTER,A,A.1
0,{79A74E21-D11F-1289-E053-6B04A8C01627},253500,2018-09-24 00:00,M6 8GQ,D,N,F,1,NaN,RIVINGTON ROAD,NaN,SALFORD,SALFORD,GREATER MANCHESTER,A,A
1,{79A74E21-D120-1289-E053-6B04A8C01627},231950,2018-09-28 00:00,WA3 2UE,D,Y,F,35,NaN,STONEACRE CLOSE,LOWTON,WARRINGTON,WIGAN,GREATER MANCHESTER,A,A
2,{79A74E21-D121-1289-E053-6B04A8C01627},112500,2018-08-29 00:00,OL6 6RJ,S,N,F,102,NaN,THORNFIELD GROVE,NaN,ASHTON-UNDER-LYNE,TAMESIDE,GREATER MANCHESTER,A,A
3,{79A74E21-D122-1289-E053-6B04A8C01627},184995,2018-06-15 00:00,M46 0TW,S,Y,F,37,NaN,THREADNEEDLE PLACE,ATHERTON,MANCHESTER,WIGAN,GREATER MANCHESTER,A,A
4,{79A74E21-D123-1289-E053-6B04A8C01627},214995,2018-09-28 00:00,M28 3XS,D,Y,L,9,NaN,MARPLE GARDENS,WORSLEY,MANCHESTER,SALFORD,GREATER MANCHESTER,A,A
5,{79A74E21-D124-1289-E053-6B04A8C01627},378000,2018-09-21 00:00,M45 7QN,D,N,L,36,NaN,HILLINGDON ROAD,WHITEFIELD,MANCHESTER,MANCHESTER,GREATER MANCHESTER,A,A
6,{79A74E21-D125-1289-E053-6B04A8C01627},272500,2018-05-25 00:00,WN6 0GQ,D,Y,L,11,NaN,HEDGEBANK,STANDISH,WIGAN,WIGAN,GREATER MANCHESTER,A,A
7,{79A74E21-D127-1289-E053-6B04A8C01627},169950,2018-09-20 00:00,WN6 0ZJ,S,Y,F,50,NaN,FOXTAIL MEADOW,STANDISH,WIGAN,WIGAN,GREATER MANCHESTER,A,A
8,{79A74E21-D128-1289-E053-6B04A8C01627},260995,2018-08-30 00:00,WN6 0UU,D,Y,F,49,NaN,STABLEFORD CLOSE,STANDISH,WIGAN,WIGAN,GREATER MANCHESTER,A,A
9,{79A74E21-D129-1289-E053-6B04A8C01627},183000,2018-08-29 00:00,M34 2WZ,S,N,L,28,NaN,HULME ROAD,DENTON,MANCHESTER,TAMESIDE,GREATER MANCHESTER,A,A


In [5]:
df_ppd.shape

(1029749, 16)

In [6]:
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [7]:
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [8]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()

In [9]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [10]:
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

In [11]:
df_affordable

,Street,Avg_Price
196,ALBION SQUARE,2.450000e+06
390,ANHALT ROAD,2.435000e+06
405,ANSDELL TERRACE,2.250000e+06
422,APPLEGARTH ROAD,2.400000e+06
855,BARONSMEAD ROAD,2.375000e+06
981,BEAUCLERC ROAD,2.480000e+06
1102,BELVEDERE DRIVE,2.340000e+06
1215,BICKENHALL STREET,2.208500e+06
1253,BIRCHLANDS AVENUE,2.217000e+06
1553,BRAMPTON GROVE,2.456875e+06


In [12]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [13]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 196
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 196, dtype: object
item.Street only: ALBION SQUARE
index: 390
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 390, dtype: object
item.Street only: ANHALT ROAD
index: 405
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 405, dtype: object
item.Street only: ANSDELL TERRACE
index: 422
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 422, dtype: object
item.Street only: APPLEGARTH ROAD
index: 855
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 855, dtype: object
item.Street only: BARONSMEAD ROAD
index: 981
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 981, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1102
item: Street       BELVEDERE DRIVE
Avg_Price           2.34e+06
Name: 1102, dtype: object
item.Street only: BELVEDERE DRIVE
index: 1215
item: Street       BICKENHALL STREET
Avg_Price           2.2085e+06
N

In [22]:
geolocator = Nominatim(user_agent="my")

In [23]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [24]:
df_affordable

,Street,Avg_Price,city_coord
196,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.28939323910353)"
390,ANHALT ROAD,2.435000e+06,"(51.4803164, -0.1668011)"
405,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)"
422,APPLEGARTH ROAD,2.400000e+06,"(53.7486539, -0.3266704)"
855,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)"
981,BEAUCLERC ROAD,2.480000e+06,"(30.2114523, -81.6179807)"
1102,BELVEDERE DRIVE,2.340000e+06,"(38.0728178, -78.4587964)"
1215,BICKENHALL STREET,2.208500e+06,"(51.5212014, -0.1589082)"
1253,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)"
1553,BRAMPTON GROVE,2.456875e+06,"(51.5899607, -0.3185249)"


In [25]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [26]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
196,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.28939323910353)",-41.273758,173.289393
390,ANHALT ROAD,2.435000e+06,"(51.4803164, -0.1668011)",51.480316,-0.166801
405,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)",51.499890,-0.189103
422,APPLEGARTH ROAD,2.400000e+06,"(53.7486539, -0.3266704)",53.748654,-0.326670
855,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)",51.477315,-0.239457
981,BEAUCLERC ROAD,2.480000e+06,"(30.2114523, -81.6179807)",30.211452,-81.617981
1102,BELVEDERE DRIVE,2.340000e+06,"(38.0728178, -78.4587964)",38.072818,-78.458796
1215,BICKENHALL STREET,2.208500e+06,"(51.5212014, -0.1589082)",51.521201,-0.158908
1253,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)",51.448394,-0.160468
1553,BRAMPTON GROVE,2.456875e+06,"(51.5899607, -0.3185249)",51.589961,-0.318525


In [27]:
df = df_affordable.drop(columns=['city_coord'])

In [28]:
df

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
390,ANHALT ROAD,2.435000e+06,51.480316,-0.166801
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
422,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670
855,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457
981,BEAUCLERC ROAD,2.480000e+06,30.211452,-81.617981
1102,BELVEDERE DRIVE,2.340000e+06,38.072818,-78.458796
1215,BICKENHALL STREET,2.208500e+06,51.521201,-0.158908
1253,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468
1553,BRAMPTON GROVE,2.456875e+06,51.589961,-0.318525


In [29]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London City are 51.5073219, -0.1276474.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


In [30]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [31]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'J0K3MZ1HXFRGBOQHE4RRRJCFFSJKXKP4CIM11SMR2PUBGIUT' # Foursquare ID
CLIENT_SECRET = 'GJHPGMB2MTPINXLEXQT00LVTR2ZWJP4HIYEHTYNHAO0UXRFD' # Foursquare Secret
VERSION = '20203105' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: J0K3MZ1HXFRGBOQHE4RRRJCFFSJKXKP4CIM11SMR2PUBGIUT
CLIENT_SECRET:GJHPGMB2MTPINXLEXQT00LVTR2ZWJP4HIYEHTYNHAO0UXRFD


In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
BARONSMEAD ROAD
BEAUCLERC ROAD
BELVEDERE DRIVE
BICKENHALL STREET
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BROOKWAY
BURBAGE ROAD
BURY WALK
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHEYNE ROW
CHISWICK MALL
CITY ROAD
CLARENDON STREET
CLONCURRY STREET
COLBECK MEWS
COLLEGE CRESCENT
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRESCENT GROVE
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
ELM PARK ROAD
FLORAL STREET
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIRDLERS ROAD
GLOUCESTER CRESCENT
GORDON PLACE
GRAFTON SQUARE
GRAHAM TERRACE
HARMAN DRIVE
HARRIS STREET
HAVANNAH STREET
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HIGHGATE HIGH STREET
HIGHWOOD HILL
HILLGATE PLACE
HOLLYCROFT AVENUE
HOLLYWOOD MEWS
HONEYWELL

In [34]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
2,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
3,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
4,ALBION SQUARE,-41.273758,173.289393,Fish Stop,-41.276010,173.289592,Fish & Chips Shop
5,ALBION SQUARE,-41.273758,173.289393,Deville Cafe,-41.271941,173.285535,Beer Garden
6,ALBION SQUARE,-41.273758,173.289393,Fresh Choice,-41.272194,173.287218,Supermarket
7,ALBION SQUARE,-41.273758,173.289393,The Bridge Street Collective,-41.272520,173.285517,Café
8,ALBION SQUARE,-41.273758,173.289393,Mango,-41.274460,173.285345,Indian Restaurant
9,ALBION SQUARE,-41.273758,173.289393,Hopgood's,-41.274749,173.283831,Restaurant


In [35]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,28,28,28,28,28,28
ANHALT ROAD,16,16,16,16,16,16
ANSDELL TERRACE,47,47,47,47,47,47
APPLEGARTH ROAD,4,4,4,4,4,4
BARONSMEAD ROAD,14,14,14,14,14,14
BEAUCLERC ROAD,6,6,6,6,6,6
BELVEDERE DRIVE,2,2,2,2,2,2
BICKENHALL STREET,60,60,60,60,60,60
BIRCHLANDS AVENUE,11,11,11,11,11,11


In [36]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 338 uniques categories.


In [37]:
location_venues.shape

(4398, 7)

In [38]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,...,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,ALBION SQUARE,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,ANHALT ROAD,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,ANSDELL TERRACE,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,APPLEGARTH ROAD,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,BARONSMEAD ROAD,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
5,BEAUCLERC ROAD,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
6,BELVEDERE DRIVE,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
7,BICKENHALL STREET,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.016667,0.00,0.000000,0.0,0.016667,0.000000,0.016667,0.016667,0.000000
8,BIRCHLANDS AVENUE,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
9,BRAMPTON GROVE,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [40]:
london_grouped.shape

(150, 339)

In [41]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
               venue  freq
0               Café  0.21
1        Coffee Shop  0.07
2                Pub  0.07
3  Indian Restaurant  0.07
4         Restaurant  0.07


----ANHALT ROAD----
               venue  freq
0                Pub  0.25
1  French Restaurant  0.12
2      Grocery Store  0.12
3              Diner  0.06
4             Garden  0.06


----ANSDELL TERRACE----
            venue  freq
0      Restaurant  0.09
1  Clothing Store  0.09
2       Juice Bar  0.06
3             Pub  0.06
4           Hotel  0.06


----APPLEGARTH ROAD----
               venue  freq
0                Pub  0.25
1             Casino  0.25
2     Sandwich Place  0.25
3          Nightclub  0.25
4  Outdoor Sculpture  0.00


----BARONSMEAD ROAD----
             venue  freq
0              Pub  0.07
1      Sports Club  0.07
2             Park  0.07
3  Nature Preserve  0.07
4   Farmers Market  0.07


----BEAUCLERC ROAD----
             venue  freq
0              Spa  0.33
1  Doctor's Office  0.1

                        venue  freq
0              Cosmetics Shop  0.25
1  Construction & Landscaping  0.25
2               Deli / Bodega  0.25
3                Home Service  0.25
4           Outdoor Sculpture  0.00


----FRANK DIXON WAY----
         venue  freq
0         Café  0.17
1         Park  0.17
2   Playground  0.17
3         Farm  0.17
4  Rugby Pitch  0.17


----FULTON MEWS----
                venue  freq
0               Hotel  0.18
1                 Pub  0.08
2         Coffee Shop  0.07
3  Chinese Restaurant  0.06
4                Café  0.06


----GERARD ROAD----
               venue  freq
0  Indian Restaurant   0.2
1  Convenience Store   0.2
2   Business Service   0.2
3     Scenic Lookout   0.2
4  Fish & Chips Shop   0.2


----GERRARD ROAD----
                  venue  freq
0                   Pub   0.5
1  Fast Food Restaurant   0.5
2                   ATM   0.0
3   Outdoor Event Space   0.0
4                  Park   0.0


----GIRDLERS ROAD----
                venue  freq
0  

                venue  freq
0                 Pub  0.33
1   Convenience Store  0.33
2      Breakfast Spot  0.33
3  Italian Restaurant  0.00
4        Perfume Shop  0.00


----MUNDEN STREET----
               venue  freq
0                Pub  0.11
1              Hotel  0.09
2  Indian Restaurant  0.09
3               Café  0.09
4           Platform  0.05


----NORTH CIRCULAR ROAD----
         venue  freq
0         Park   0.2
1  Supermarket   0.2
2          Pub   0.2
3        Hotel   0.2
4     Bus Stop   0.2


----NOTTINGHAM STREET----
                  venue  freq
0   Sporting Goods Shop  0.25
1                Bakery  0.25
2             Wine Shop  0.25
3                 Trail  0.25
4  Other Great Outdoors  0.00


----OAKLEY STREET----
                  venue  freq
0                Lounge   0.5
1                   Bar   0.5
2       Nature Preserve   0.0
3  Outdoor Supply Store   0.0
4      Pedestrian Plaza   0.0


----OAKWOOD COURT----
         venue  freq
0         Food  0.25
1  Golf Cour

                venue  freq
0                 Pub  0.31
1              Bakery  0.06
2   Indian Restaurant  0.06
3                Café  0.06
4  Italian Restaurant  0.06


----SOVEREIGN PARK----
                venue  freq
0  Chinese Restaurant   0.2
1                 Gym   0.2
2         Pizza Place   0.2
3                Café   0.2
4                 Bar   0.2


----ST OSWALDS PLACE----
                venue  freq
0                 Pub  0.12
1                Café  0.10
2  Italian Restaurant  0.07
3             Gay Bar  0.07
4                Park  0.05


----ST PETERS SQUARE----
                venue  freq
0  Italian Restaurant  0.23
1      Ice Cream Shop  0.10
2               Hotel  0.08
3                Café  0.06
4      Sandwich Place  0.04


----STAFFORD TERRACE----
                  venue  freq
0           Supermarket  0.33
1                   Pub  0.11
2           Pizza Place  0.11
3   Rental Car Location  0.11
4  Fast Food Restaurant  0.11


----SUTHERLAND PLACE----
               

In [42]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [44]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

In [45]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Café,Bar,Indian Restaurant,Pub,Restaurant,Coffee Shop,Park,Museum,Fish & Chips Shop,French Restaurant
1,ANHALT ROAD,Pub,Grocery Store,French Restaurant,Cocktail Bar,Garden,English Restaurant,Gym / Fitness Center,Diner,Convenience Store,Plaza
2,ANSDELL TERRACE,Restaurant,Clothing Store,Hotel,Pub,Juice Bar,Bakery,Italian Restaurant,Indian Restaurant,Department Store,Supermarket
3,APPLEGARTH ROAD,Sandwich Place,Casino,Pub,Nightclub,Farm,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit
4,BARONSMEAD ROAD,Indie Movie Theater,Thai Restaurant,Pub,Coffee Shop,Restaurant,Park,Café,Farmers Market,Nature Preserve,Breakfast Spot


In [46]:
venues_sorted.shape

(150, 11)

In [47]:
london_grouped.shape

(150, 339)

In [48]:
london_grouped=df

In [49]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([1, 3, 0, 3, 2, 1, 2, 0, 0, 1, 3, 3, 3, 1, 2, 2, 1, 3, 0, 1, 4, 4,
       3, 1, 1, 0, 3, 4, 1, 0, 3, 2, 3, 2, 2, 4, 3, 3, 2, 0, 1, 2, 4, 0,
       4, 0, 0, 4, 0, 0], dtype=int32)

In [50]:
#Dataframe to include Clusters

london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2450000.0,-41.273758,173.289393
390,ANHALT ROAD,2435000.0,51.480316,-0.166801
405,ANSDELL TERRACE,2250000.0,51.499890,-0.189103
422,APPLEGARTH ROAD,2400000.0,53.748654,-0.326670
855,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457


In [51]:
london_grouped_clustering.shape

(162, 4)

In [52]:
df.shape

(162, 4)

In [53]:
london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [54]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [55]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,1,Café,Bar,Indian Restaurant,Pub,Restaurant,Coffee Shop,Park,Museum,Fish & Chips Shop,French Restaurant
390,ANHALT ROAD,2.435000e+06,51.480316,-0.166801,3,Pub,Grocery Store,French Restaurant,Cocktail Bar,Garden,English Restaurant,Gym / Fitness Center,Diner,Convenience Store,Plaza
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103,0,Restaurant,Clothing Store,Hotel,Pub,Juice Bar,Bakery,Italian Restaurant,Indian Restaurant,Department Store,Supermarket
422,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670,3,Sandwich Place,Casino,Pub,Nightclub,Farm,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit
855,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,2,Indie Movie Theater,Thai Restaurant,Pub,Coffee Shop,Restaurant,Park,Café,Farmers Market,Nature Preserve,Breakfast Spot
981,BEAUCLERC ROAD,2.480000e+06,30.211452,-81.617981,1,Spa,Pizza Place,Automotive Shop,Harbor / Marina,Doctor's Office,Zoo,Farm,Ethiopian Restaurant,Event Space,Exhibit
1102,BELVEDERE DRIVE,2.340000e+06,38.072818,-78.458796,2,Pool,Playground,Farmers Market,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant
1215,BICKENHALL STREET,2.208500e+06,51.521201,-0.158908,0,Gastropub,Restaurant,Hotel,Café,Italian Restaurant,Pizza Place,Garden,Bar,Movie Theater,Greek Restaurant
1253,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468,0,Pub,French Restaurant,Bakery,Train Station,Pizza Place,Brewery,Chinese Restaurant,Coffee Shop,Lake,Ethiopian Restaurant
1553,BRAMPTON GROVE,2.456875e+06,51.589961,-0.318525,1,Construction & Landscaping,Zoo,Fast Food Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm


In [56]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [57]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
405,2250000.0,Restaurant,Clothing Store,Hotel,Pub,Juice Bar,Bakery,Italian Restaurant,Indian Restaurant,Department Store,Supermarket
1215,2208500.0,Gastropub,Restaurant,Hotel,Café,Italian Restaurant,Pizza Place,Garden,Bar,Movie Theater,Greek Restaurant
1253,2217000.0,Pub,French Restaurant,Bakery,Train Station,Pizza Place,Brewery,Chinese Restaurant,Coffee Shop,Lake,Ethiopian Restaurant
2225,2200000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2638,2250000.0,Pharmacy,Cosmetics Shop,Hotel,Pizza Place,Bakery,Grocery Store,Bookstore,Coffee Shop,Supermarket,Falafel Restaurant


In [58]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,2450000.0,Café,Bar,Indian Restaurant,Pub,Restaurant,Coffee Shop,Park,Museum,Fish & Chips Shop,French Restaurant
981,2480000.0,Spa,Pizza Place,Automotive Shop,Harbor / Marina,Doctor's Office,Zoo,Farm,Ethiopian Restaurant,Event Space,Exhibit
1553,2456875.0,Construction & Landscaping,Zoo,Fast Food Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
1980,2492500.0,Supermarket,English Restaurant,Gym,Fast Food Restaurant,Café,Rental Car Location,Coffee Shop,Hardware Store,Pub,American Restaurant
2136,2461000.0,Food,Trail,Zoo,Farm,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory


In [59]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
855,2375000.0,Indie Movie Theater,Thai Restaurant,Pub,Coffee Shop,Restaurant,Park,Café,Farmers Market,Nature Preserve,Breakfast Spot
1102,2340000.0,Pool,Playground,Farmers Market,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant
2068,2375000.0,Pub,Park,Ice Cream Shop,Grocery Store,Yoga Studio,Indian Restaurant,Bakery,Hotel,Burger Joint,Gastropub
2129,2379652.7,Pub,Grocery Store,Bakery,Indian Restaurant,Coffee Shop,Park,Yoga Studio,Hotel,Ice Cream Shop,Movie Theater
2944,2367500.0,Hotel,Pub,Garden,Coffee Shop,Café,Italian Restaurant,Bar,Mediterranean Restaurant,Chinese Restaurant,Gym / Fitness Center


In [60]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
390,2435000.0,Pub,Grocery Store,French Restaurant,Cocktail Bar,Garden,English Restaurant,Gym / Fitness Center,Diner,Convenience Store,Plaza
422,2400000.0,Sandwich Place,Casino,Pub,Nightclub,Farm,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit
1632,2397132.0,Grocery Store,Gym / Fitness Center,Italian Restaurant,Coffee Shop,Health & Beauty Service,Falafel Restaurant,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant
1797,2400000.0,Home Service,Art Gallery,Zoo,Fast Food Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant
1914,2445000.0,Construction & Landscaping,Grocery Store,Dance Studio,Bar,Athletics & Sports,Farmers Market,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit


In [61]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2242,2.300000e+06,Construction & Landscaping,Farm,Zoo,Fast Food Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant
2406,2.286679e+06,Café,Pub,Bar,Italian Restaurant,Coffee Shop,Park,Convenience Store,French Restaurant,Beer Store,Greek Restaurant
2686,2.287500e+06,Pub,Brewery,Gym / Fitness Center,Gift Shop,Art Museum,Zoo,Farm,English Restaurant,Ethiopian Restaurant,Event Space
3377,2.298000e+06,Hotel,Zoo,Fast Food Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm
4285,2.265000e+06,American Restaurant,Gym,Dry Cleaner,Trail,Farm,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit
